This Notebook is my first ever attempt at a Kaggle competition. It has consistently produced submission scores in the vicinity of 0.8000 and the best rank I have achieved on the leaderboard so far is 379. For other learners like me, I think this notebook may also come in handy as a 'solved example' that covers a host of concepts such as basic data analysis, data cleaning, pre-processing steps including imputation, scaling, encoding, dimensionality reduction, correlation / mutual-info matrix, and multiple classification algorithms including ensemble models, and deep learning / neural networks, and also ways to maximize training data utilization while still minimizing overfitting / generalization error. 

If you found this interesting or learned something from it, request to please Upvote this Notebook. And of course, I will appreciate any feedback / constructive criticism. Thanks very much. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
# Import Libraries, classes, and functions 
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler 
from sklearn.impute import KNNImputer
from sklearn.feature_selection import mutual_info_regression 
from sklearn.decomposition import PCA, TruncatedSVD  
from sklearn.compose import make_column_transformer 
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, KFold, StratifiedKFold, StratifiedShuffleSplit  
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier  
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import GaussianNB 
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score 

from xgboost import XGBClassifier 
from lightgbm import LGBMClassifier 

# Set random seed 
SEED = np.random.default_rng().integers(99999)
# SEED = 999999 
from tensorflow.keras.utils import set_random_seed 
set_random_seed = SEED


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
# Import training dataset into padas DataFrame & explore 
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv') 

print(train.head(), '\n', 
train.info(), '\n', 
train.describe(include='all').T, '\n', 
train.nunique() , '\n', 
train.isna().sum() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   


In [4]:
# Import test dataset into padas DataFrame & explore 
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv') 

print(test.head(), '\n', 
test.info(), '\n', 
test.describe(include='all').T, '\n', 
test.nunique() , '\n', 
test.isna().sum() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB
  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0013_01      Earth      True  G/3/S  TRAPPIST-1e  27.0  False   
1     0018_01      Earth     False  F/4/S  TRAPPIST-

In [5]:
# merge test and train datasets into single dataframe for data cleaning and necessary pre-processing 
all = pd.concat([train, test], axis=0) 

print(all.head(), '\n', 
all.info(), '\n', 
all.describe(include='all').T, '\n', 
all.nunique() , '\n', 
all.isna().sum() )

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12970 entries, 0 to 4276
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   12970 non-null  object 
 1   HomePlanet    12682 non-null  object 
 2   CryoSleep     12660 non-null  object 
 3   Cabin         12671 non-null  object 
 4   Destination   12696 non-null  object 
 5   Age           12700 non-null  float64
 6   VIP           12674 non-null  object 
 7   RoomService   12707 non-null  float64
 8   FoodCourt     12681 non-null  float64
 9   ShoppingMall  12664 non-null  float64
 10  Spa           12686 non-null  float64
 11  VRDeck        12702 non-null  float64
 12  Name          12676 non-null  object 
 13  Transported   8693 non-null   object 
dtypes: float64(6), object(8)
memory usage: 1.5+ MB
  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002

In [6]:
# drop duplicate rows 
all.drop_duplicates(keep='first', inplace=True) 
print(all.dtypes)

# Feature Engineering 
# split PassengerId column into its component parts (group, and id ) as separate features  
all[['group', 'id']] = all['PassengerId'].str.split(pat='_', expand=True) 

# split Cabin column into its component parts (deck, cabin id, side ) as separate features 
all[['deck', 'cabin_no', 'side']] = all.Cabin.str.split('/', expand=True) 

# split Name column into its component parts (first and last name) as separate features
all[['FName', 'LName']] = all.Name.str.split(" ", expand=True) 
''' The intuition behind splitting name is that family name or first name may potentially 
have some influence on who gets transported into the anomaly '''

# Treatment of High-cardinality columns : columns that have too many categories (eg. upwards of 25 categories) 
''' add value_counts column for high cardinality columns & then drop original col, and treat resulting values as numerical data 
alternatively, transform high_cardinality column data into numbers (ordinal encoding) and treat as numerical data '''

# drop unique value columns  
print(all.columns)

# Subsetting columns for pre-processing with relevant encoders 
''' Since most machine learning models will require all data in numerical form, we will use relevant 
encoders to transform our data accordingly. Numerical columns will not undergo any encoding. 
Columns with ordinal values (ranks, value_counts, high-cardinality columns, etc) will be processed 
using Ordinal Encoder. Columns with categorical values (low-cardinality unique un-ordered values) 
to be OneHotEncoded, but we will first convert them into numerical codes (with Ordinal Encoder) so 
the whole dataset is already numerical for missing value imputation using an Imputation algorithms, 
eg. KNNImputer we have used here. '''

numeric = ['Age', 'group', 'id', 'cabin_no', 'RoomService', 'FoodCourt', 'ShoppingMall','Spa','VRDeck',] 
ordinal = ['FName', 'LName'] 
categorical = ['HomePlanet', 'Destination', 'deck', 'side', 'CryoSleep', 'VIP', 'Transported'] 

# checking unique values in each categorical columns to address 
# any mistakes (such as due to spelling, uppercase-lowercase or similar errors,)
uniq_cats = {col: sorted(all[col].dropna().unique().tolist() ) for col in categorical} 
# print(uniq_cats) 

# Converting target column booleans to corresponding Integer values 
all.Transported = all.Transported.replace({True: 1, False:0})

# Viewing summary statistics for all columns to check for any un-natural (eg. negative values where there should be none, etc) values,
print(all.describe(include='all').T)

# converting into numeric dtypes where numbers are appearing as object datatype 
all[['group', 'id', 'cabin_no']] = all[['group', 'id', 'cabin_no']].apply(pd.to_numeric, axis=0)
print(all.info())

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported      object
dtype: object
Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported', 'group', 'id', 'deck', 'cabin_no', 'side',
       'FName', 'LName'],
      dtype='object')
                count unique               top   freq        mean  \
PassengerId     12970  12970           0001_01      1         NaN   
HomePlanet      12682      3             Earth   6865         NaN   
CryoSleep       12660      2             False   8079         NaN   
Cabin           12671   9825           G/734/S      8         NaN   
Destination     12696      3 

In [7]:
# dropping columns that have been split into their component values 
# setting original PassengerId column as row index 
all.set_index('PassengerId', inplace=True) 

# dropping Cabin & Name columns that were split previously 
all.drop(['Cabin', 'Name'], axis=1, inplace=True)

# checking all columns in our dataset match with those covered under column groupings
print(len(all.columns), len(numeric + ordinal + categorical) )

# Transforming all NA value types (pandas, numpy NA types, etc) into np.nan type for uniformity 
all = all.fillna(np.nan) 

# Splitting our merged dataset into original train and test sets, 
# where original Test Set had missing ('NA') values in Transported column 
X_train = all[all.Transported.notna()].drop('Transported', axis=1)
y_train = all[all.Transported.notna()].Transported.astype(int) 
X_test = all[all.Transported.isna()].drop('Transported', axis=1)

# deleting the target column name from our categorical columns grouping list 
categorical.remove('Transported')

# checking for data imbalance
print('Mean of target classes: ', y_train.mean())

''' Since this is a binary classification problem, calculating mean value of target column will suffice. 
Here it is 0.503 indicating that the data very well balanced into positive and negative target classes. 
In case we had severely unbalanced data between target classes (say 0.75:0.25), we may have to address 
the imbalance, for example, by bootstrapping the minority class column to match the sample size of the 
majority class, or alternatively sampling from majority class column to match sample size of minority class''' 


18 18
Mean of target classes:  0.5036236051995858


' Since this is a binary classification problem, calculating mean value of target column will suffice. \nHere it is 0.503 indicating that the data very well balanced into positive and negative target classes. \nIn case we had severely unbalanced data between target classes (say 0.75:0.25), we may have to address \nthe imbalance, for example, by bootstrapping the minority class column to match the sample size of the \nmajority class, or alternatively sampling from majority class column to match sample size of minority class'

In [8]:
# Using encoders on column groupings to convert data into numerical values where applicable 
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan) 

''' we will fit the encoders to training data and transform both training and test datasets using the 
same patterns learned from training data. This is to avoid leakage (of insights from test data (which 
is not available at the time of training in real world application) into training dataset) '''
X_train[ordinal + categorical] = ordinal_encoder.fit_transform(X_train[ordinal + categorical]) 
X_test[ordinal + categorical] = ordinal_encoder.transform(X_test[ordinal + categorical])

# check dtypes for each column and correct where necessary 
print(X_train.info(), y_train.info(), X_test.info(), sep = '\n\n')
print(X_train.describe(), y_train.describe(), X_test.describe(), sep = '\n\n')


<class 'pandas.core.frame.DataFrame'>
Index: 8693 entries, 0001_01 to 9280_02
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   float64
 1   CryoSleep     8476 non-null   float64
 2   Destination   8511 non-null   float64
 3   Age           8514 non-null   float64
 4   VIP           8490 non-null   float64
 5   RoomService   8512 non-null   float64
 6   FoodCourt     8510 non-null   float64
 7   ShoppingMall  8485 non-null   float64
 8   Spa           8510 non-null   float64
 9   VRDeck        8505 non-null   float64
 10  group         8693 non-null   int64  
 11  id            8693 non-null   int64  
 12  deck          8494 non-null   float64
 13  cabin_no      8494 non-null   float64
 14  side          8494 non-null   float64
 15  FName         8493 non-null   float64
 16  LName         8493 non-null   float64
dtypes: float64(15), int64(2)
memory usage: 1.2+ MB
<class 'pandas.core.

In [9]:
# Scaling data before using missing value Imputation algorithm 
'''We are going to use KNN Imputer to impute missing values in our train and test datasets. 
Since KNN algorithm is informed by variances or deviation between datapoints, it is important that we first 
scale all data to a uniform scale. Now, since we want to later One-Hot-Encode the categorical 
columns which will require unique (integer) values, we cannot scale the categorical columns 
as that will turn them into float values. So, we will instead take the min and max values across
all categorical clumns (calculates to 0 and 7 in this case) and scale the remaining data accordingly 
using MinMaxScaler '''

# calculate min & max values for all categorical features combined 
min_cat = X_train[categorical].min().min()
max_cat = X_train[categorical].max().max()
print('Min-Max spread of values in categorical columns: \n', min_cat, max_cat) 

# Scaling using MinMaxScaler 
mmScaler = MinMaxScaler(feature_range=(min_cat, max_cat)) 
X_train[numeric + ordinal] = mmScaler.fit_transform(X_train[numeric + ordinal]) 
X_test[numeric + ordinal] = mmScaler.transform(X_test[numeric + ordinal]) 

# checking column-wise standard deviation for scaled data 
print('column-wise standard deviation in Training set after min-max scaling')
print(X_train.std(axis=0).T)
print('column-wise standard deviation in Test set after min-max scaling')
print(X_test.std(axis=0).T)

Min-Max spread of values in categorical columns: 
 0.0 7.0
column-wise standard deviation in Training set after min-max scaling
HomePlanet      0.798155
CryoSleep       0.479531
Destination     0.820237
Age             1.283837
VIP             0.151303
RoomService     0.325750
FoodCourt       0.378373
ShoppingMall    0.180184
Spa             0.355094
VRDeck          0.332326
group           2.015002
id              1.054241
deck            1.778233
cabin_no        1.891801
side            0.500006
FName           1.982338
LName           2.026342
dtype: float64
column-wise standard deviation in Test set after min-max scaling
HomePlanet      0.811815
CryoSleep       0.482598
Destination     0.807489
Age             1.256373
VIP             0.131825
RoomService     0.296578
FoodCourt       0.358691
ShoppingMall    0.167110
Spa             0.348996
VRDeck          0.361702
group           2.049077
id              1.018221
deck            1.719892
cabin_no        1.903261
side            0

In [10]:
# Using KNN Imputer to impute missing values 

# ensure all NA values are represented uniformluy as np.nan 
X_train = X_train.fillna(np.nan)
X_test = X_test.fillna(np.nan) 

# Instantiate KNNImputer using default 5 neighbors 
imputer = KNNImputer(n_neighbors=5) 

'''KNN Imputer will return numpy arrays. In order to retain our dataframe with column names 
(to use for further preprocessig), we will assign the numpy output to our pandas dataframes 
using .loc[:] accessor for all data. '''

X_train.loc[:] = imputer.fit_transform(X_train) 
X_test.loc[:] = imputer.transform(X_test) 

# Checking if any missing values still remain 
# print(X_train.isna().sum().T) 
# print(X_test.isna().sum().T) 


In [11]:
# Calculating correlation (linear) and mutual-info (non-linear) of dataset columns / features with target column 
correlation = pd.concat([X_train, y_train], axis=1).corr().Transported.abs().values[:-1]
mutual_info = mutual_info_regression(X_train, y_train, random_state=SEED) 
associations = {'correlation': correlation, 'mutual_info': mutual_info} 
associations = pd.DataFrame(associations, index=X_train.columns).sort_values(by='correlation', ascending=False)
display(associations.style.background_gradient() )

''' Instead of drawing multiple visualizations to see how various features interact with 
each other and with the target variable, I prefer to use a correlation / mutual-info matrix that gives me 
a snapshot of interactions in one tabular view. Looking at the results of both linear correlation
and non-linear mutual-info regression, it appears Cryosleep status had the maximum influence 
on who was Transported into the anomaly'''

# Checking summary statistics after missing value imputation 
print('Summary statistics after missing value imputation: ') 
print(X_train.info(), X_train.describe().T, sep='\n') 
print(X_test.info(), X_test.describe().T, sep='\n') 

,correlation,mutual_info
CryoSleep,0.464822,0.110660
RoomService,0.243517,0.065398
Spa,0.218882,0.059129
VRDeck,0.206177,0.072151
HomePlanet,0.119572,0.023737
deck,0.117175,0.020142
Destination,0.109219,0.011382
side,0.103062,0.008276
Age,0.072878,0.017392
id,0.066390,0.009302


Summary statistics after missing value imputation: 
<class 'pandas.core.frame.DataFrame'>
Index: 8693 entries, 0001_01 to 9280_02
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8693 non-null   float64
 1   CryoSleep     8693 non-null   float64
 2   Destination   8693 non-null   float64
 3   Age           8693 non-null   float64
 4   VIP           8693 non-null   float64
 5   RoomService   8693 non-null   float64
 6   FoodCourt     8693 non-null   float64
 7   ShoppingMall  8693 non-null   float64
 8   Spa           8693 non-null   float64
 9   VRDeck        8693 non-null   float64
 10  group         8693 non-null   float64
 11  id            8693 non-null   float64
 12  deck          8693 non-null   float64
 13  cabin_no      8693 non-null   float64
 14  side          8693 non-null   float64
 15  FName         8693 non-null   float64
 16  LName         8693 non-null   float64
dtypes: float64(17)


In [12]:
# Transforming Categorical features using OneHotEncoder 
print('Before OHE', X_train.shape, X_test.shape, sep='\n')
onehot = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False) 
onehot = make_column_transformer((onehot, categorical), remainder='passthrough')
X_train = onehot.fit_transform(X_train)
X_test = onehot.transform(X_test) 
print('After OHE', X_train.shape, X_test.shape, sep='\n')

# # Dimensionality Reduction using PCA 
# pca = PCA(n_components=0.99, random_state=SEED) 
# X_train = pca.fit_transform(X_train) 
# X_test = pca.transform(X_test) 
# print('After PCA', X_train.shape, X_test.shape, sep='\n')
# ''' PCA is adversely affecting performance in all my trials with it, so excluding it ''' 

# scaler = StandardScaler(with_mean=True)
scaler = RobustScaler(with_centering=True)
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 

# Retaining full training set for training model to be used for submission  
X_train_full, y_train_full = X_train.copy(), y_train.copy()

# Creating training and validation split (including corresponding indices from the base set) to evaluate model performances 
X_train, X_val, y_train, y_val, tts_tr, tts_val = train_test_split(X_train_full, y_train_full, np.arange(X_train_full.shape[0]), 
                                                                   test_size=0.20, stratify=y_train_full, random_state=SEED) 
print([i.shape[0] for i in [X_train, X_val, y_train, y_val, tts_tr, tts_val] ]) 
print('SEED: ', SEED)

Before OHE
(8693, 17)
(4277, 17)
After OHE
(8693, 69)
(4277, 69)
[6954, 1739, 6954, 1739, 6954, 1739]
SEED:  66451


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [0, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [13]:
# # exploring pipelines, including algorithms, using TPOT library 
# from tpot import TPOTClassifier 
# tpot = TPOTClassifier(generations=None, population_size=7, verbosity=2, offspring_size=10, 
#                       scoring='neg_log_loss', cv=5, n_jobs=-1, max_time_mins=10, ) 
# tpot.fit(X_train, y_train) 
# print(tpot.score(X_train, y_train))

''' TPOT returned various tuned versions of RandomForestClassifier, none of which gave a competition 
accuuracy score of more than 77% (much less than the score achieved by other models in this Notebook) 
so I didn't pursue it any further. 

For reference, some of the pipelines & model returned by various TPOT runs (model parameters with their 
corresponding neg_log_loss score) are as follows:

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=False, criterion=entropy, max_features=0.5, min_samples_leaf=17, min_samples_split=12, n_estimators=100)
-0.312272328007534 

Best pipeline: RandomForestClassifier(CombineDFs(Normalizer(input_matrix, norm=max), input_matrix), bootstrap=True, criterion=entropy, max_features=0.9500000000000001, min_samples_leaf=15, min_samples_split=7, n_estimators=100)
-0.3149593362816865

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=True, criterion=entropy, max_features=0.6500000000000001, min_samples_leaf=8, min_samples_split=19, n_estimators=100)
-0.29370608855794433 ''' 


" TPOT returned various tuned versions of RandomForestClassifier, none of which gave a competition \naccuuracy score of more than 77% (much less than the score achieved by other models in this Notebook) \nso I didn't pursue it any further. \n\nFor reference, some of the pipelines & model returned by various TPOT runs (model parameters with their \ncorresponding neg_log_loss score) are as follows:\n\nBest pipeline: RandomForestClassifier(input_matrix, bootstrap=False, criterion=entropy, max_features=0.5, min_samples_leaf=17, min_samples_split=12, n_estimators=100)\n-0.312272328007534 \n\nBest pipeline: RandomForestClassifier(CombineDFs(Normalizer(input_matrix, norm=max), input_matrix), bootstrap=True, criterion=entropy, max_features=0.9500000000000001, min_samples_leaf=15, min_samples_split=7, n_estimators=100)\n-0.3149593362816865\n\nBest pipeline: RandomForestClassifier(input_matrix, bootstrap=True, criterion=entropy, max_features=0.6500000000000001, min_samples_leaf=8, min_samples_sp

In [14]:
# # Modeling - Running various model configurations to find the model that gives the best log_loss score
# # I have not included keras neural networks here, because we will construct and compare those separately later 

# rs = dict(random_state=SEED)
# models = []
# models += [LogisticRegression(max_iter=200), SVC(probability=True, **rs), KNeighborsClassifier(), DecisionTreeClassifier(**rs), 
#            RandomForestClassifier(**rs), AdaBoostClassifier(**rs), GradientBoostingClassifier(**rs), MLPClassifier(max_iter=200, **rs), 
#            GaussianNB()] 
# models += [GradientBoostingClassifier(learning_rate=0.05, max_depth=7, max_features='log2', n_estimators=154, subsample=0.55, **rs), 
#            GradientBoostingClassifier(subsample=0.9, **rs), ]
# models += [RandomForestClassifier(bootstrap=False, criterion='entropy', max_features=0.5, min_samples_leaf=17, min_samples_split=12, n_estimators=100, **rs), 
#            RandomForestClassifier(bootstrap=True, criterion='entropy', max_features=0.65, min_samples_leaf=8, min_samples_split=19, n_estimators=100, **rs),
#            RandomForestClassifier(bootstrap=True, criterion='entropy', max_features=0.95, min_samples_leaf=15, min_samples_split=7, n_estimators=100, **rs)]
# models += [GradientBoostingClassifier(max_depth=7, max_features='sqrt', n_estimators=600, n_iter_no_change=50, tol=0, subsample=0.75, learning_rate=0.05, validation_fraction=0.2, verbose=0, random_state=SEED)]
# models += [XGBClassifier(**rs), LGBMClassifier(**rs), 
#            LGBMClassifier(**rs, n_estimators=600, learning_rate=0.05, metric='binary_logloss',) ]

# # Creating empty lists to record various model scores for comparison 
# logloss, accu, roc, mdl = [], [], [], [] 

# ''' Since this is a binary classification problem, I have relied on 
# log_loss / binary_crossentropy scores to compare model performances'''

# # X_val, y_val = X_train_full, y_train_full # to check score on full training set - use only when required 

# # fitting models on training set & recording prediction scores on validation set 
# for model in models:
#     if model.__class__.__name__ == 'LGBMClassifier': 
#         model.fit(X_train, y_train, early_stopping_rounds=50, eval_set=[(X_val, y_val)], verbose=0 ) 
#     else: 
#         model.fit(X_train, y_train) 
#     y_predprob_val = model.predict_proba(X_val)[:, 1] 
#     accu.append(model.score(X_val, y_val))
#     logloss.append(log_loss(y_val, y_predprob_val) )
#     roc.append(roc_auc_score(y_val, y_predprob_val))
#     mdl.append(model.__class__.__name__) 
#     print('.', end='')

# # saving scores in a dataframe for convenient viewing and analysis 
# scores_df = pd.DataFrame(dict(mdl=mdl, logloss=logloss, accuracy=accu, roc_score=roc) ) 
# print('\n', scores_df.sort_values('logloss')) 
# print('\n-Above list sorted by logloss, below is in the order of model fitting-')
# print(scores_df)

In [15]:
''' Based on the observations in our above trials, although LGBMClassifier shows similar metrics 
as GradientBoostingClassifier, in the actual submissions GradientBoostingClassifier has consistently 
outperformed other models, so I have tried to improve this algorithm via hyperparameters-tuning. We will 
continue adding more models to our lists of models to compare performance as we keep improving our models 

Following results obtained from above code (using SEED=4326) for reference
                           mdl  logloss  accuracy  roc_score
17              LGBMClassifier     0.39      0.81       0.90
16              LGBMClassifier     0.39      0.81       0.90
14  GradientBoostingClassifier     0.40      0.81       0.90
10  GradientBoostingClassifier     0.40      0.81       0.90
6   GradientBoostingClassifier     0.40      0.81       0.90
9   GradientBoostingClassifier     0.40      0.81       0.90
11      RandomForestClassifier     0.41      0.81       0.89
13      RandomForestClassifier     0.42      0.80       0.89
15               XGBClassifier     0.42      0.81       0.89
12      RandomForestClassifier     0.43      0.81       0.89
4       RandomForestClassifier     0.44      0.81       0.89
7                MLPClassifier     0.44      0.78       0.88
0           LogisticRegression     0.45      0.79       0.87
1                          SVC     0.50      0.79       0.86
5           AdaBoostClassifier     0.67      0.80       0.88
2         KNeighborsClassifier     2.10      0.77       0.84
8                   GaussianNB     5.03      0.53       0.82
3       DecisionTreeClassifier     9.49      0.74       0.74

-Above list sorted by logloss, below is in the order of model fitting-
                           mdl  logloss  accuracy  roc_score
0           LogisticRegression     0.45      0.79       0.87
1                          SVC     0.50      0.79       0.86
2         KNeighborsClassifier     2.10      0.77       0.84
3       DecisionTreeClassifier     9.49      0.74       0.74
4       RandomForestClassifier     0.44      0.81       0.89
5           AdaBoostClassifier     0.67      0.80       0.88
6   GradientBoostingClassifier     0.40      0.81       0.90
7                MLPClassifier     0.44      0.78       0.88
8                   GaussianNB     5.03      0.53       0.82
9   GradientBoostingClassifier     0.40      0.81       0.90
10  GradientBoostingClassifier     0.40      0.81       0.90
11      RandomForestClassifier     0.41      0.81       0.89
12      RandomForestClassifier     0.43      0.81       0.89
13      RandomForestClassifier     0.42      0.80       0.89
14  GradientBoostingClassifier     0.40      0.81       0.90
15               XGBClassifier     0.42      0.81       0.89
16              LGBMClassifier     0.39      0.81       0.90
17              LGBMClassifier     0.39      0.81       0.90
'''

' Based on the observations in our above trials, although LGBMClassifier shows similar metrics \nas GradientBoostingClassifier, in the actual submissions GradientBoostingClassifier has consistently \noutperformed other models, so I have tried to improve this algorithm via hyperparameters-tuning. We will \ncontinue adding more models to our lists of models to compare performance as we keep improving our models \n\nFollowing results obtained from above code (using SEED=4326) for reference\n                           mdl  logloss  accuracy  roc_score\n17              LGBMClassifier     0.39      0.81       0.90\n16              LGBMClassifier     0.39      0.81       0.90\n14  GradientBoostingClassifier     0.40      0.81       0.90\n10  GradientBoostingClassifier     0.40      0.81       0.90\n6   GradientBoostingClassifier     0.40      0.81       0.90\n9   GradientBoostingClassifier     0.40      0.81       0.90\n11      RandomForestClassifier     0.41      0.81       0.89\n13      Ran

In [16]:
# # Hyperparameters-tuning the best performing model so far - GradientBoostingClassifier 
# models = [GradientBoostingClassifier(n_estimators=300, n_iter_no_change=25, tol=0.0001,)  ] 
# ftd_model = {}
# gb_params = {"learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3], "subsample": np.arange(0.40, 1.05, 0.05), 
#              "max_depth": np.arange(4, 10), # "n_estimators": np.arange(100, 300), 
#              "max_features": ['sqrt', 'log2', None], "validation_fraction": [0.25], } 
# cv = [(tts_tr, tts_val)]
# for model in models:
#     rcv = RandomizedSearchCV(estimator=model, param_distributions=gb_params, n_iter=72, n_jobs=-1, 
#                              random_state=SEED, scoring='neg_log_loss', cv=cv, verbose=3, ) 
#     rcv.fit(X_train_full, y_train_full)
#     ftd_model[model.__class__.__name__] = rcv.best_estimator_
#     print(rcv.best_score_, rcv.best_estimator_, sep='\n\n')

# rcv_results = pd.DataFrame(rcv.cv_results_)
# rcv_results.to_csv('rcv_results.csv')

In [17]:
''' In separate runs of RandomizedSearchCV, I found following to be the best performing model configurations: 
GradientBoostingClassifier(learning_rate=0.05, max_depth=7, max_features='log2', n_estimators=154, subsample=0.55, random_state=SEED) , 

GradientBoostingClassifier(max_depth=7, max_features='sqrt', n_estimators=600, n_iter_no_change=50, tol=0, subsample=0.75, 
learning_rate=0.05, validation_fraction=0.2, verbose=0, random_state=SEED) , 

GradientBoostingClassifier(max_depth=5, max_features='sqrt', n_estimators=300, n_iter_no_change=25, tol=0, subsample=0.75, 
learning_rate=0.1, validation_fraction=0.2, verbose=0, random_state=SEED) 
'''

" In separate runs of RandomizedSearchCV, I found following to be the best performing model configurations: \nGradientBoostingClassifier(learning_rate=0.05, max_depth=7, max_features='log2', n_estimators=154, subsample=0.55, random_state=SEED) , \n\nGradientBoostingClassifier(max_depth=7, max_features='sqrt', n_estimators=600, n_iter_no_change=50, tol=0, subsample=0.75, \nlearning_rate=0.05, validation_fraction=0.2, verbose=0, random_state=SEED) , \n\nGradientBoostingClassifier(max_depth=5, max_features='sqrt', n_estimators=300, n_iter_no_change=25, tol=0, subsample=0.75, \nlearning_rate=0.1, validation_fraction=0.2, verbose=0, random_state=SEED) \n"

In [18]:
''' I have tried to minimize overfitting through multiple iterations of fitting models on separate subsamples of training 
data. 

By splitting training data into train and validation sets, and only fitting on training set, I think that we miss using for 
fitting the validation data, which is a large part of our available training data. But we cannot also use the whole training 
set for fitting because we need unseen data for validation. How do we solve this issue? For evaluating model performance, we 
do this through cross-validation algorithms(like cross_val_score), but I haven't found anything that will use 
this concept of cross-validated training for predicting on new data. So, I just tried a hacky way to achieve this. 
I split the full training set into multiple training and validation sets using StratifiedKFold, or StratifiedShuffleSplit, 
and then used the training split for fitting, validation split to minimize model loss, and then used this trained 
model to make predictions on the test set. I obtained multiple predictions on test set this way and then used 
soft-voting (averaging prediction probabilities) to get average prediction probability for each record, which I have 
eventually used in my model submission. In my trials, I noticed that soft-voting this way gave better prediction 
scores on kaggle submissions than hard-voting. This approach has worked well most of the times with the submission 
based on averages giving better kaggle score than any of the individual set of generated predictions. 
''' 

pred_dict = dict() # to save separate predictions basis shuffled splits of training data 
n_splits = 5 # no of shuffled splits of data we will try 

In [19]:
# # Exploring the Light Gradient Boosting Classifier 
# ''' LGBMClassifier has produced almost par results with our tuned sklearn GradientBoostingClassifier on my validation runs, 
# but in the Kaggle submissions, the sklearn variant has almost always produced a better score, so I have decided to run my 
# submissions using the sklearn model until something else gives me better results  '''

# # LGBMClassifier with early stopping basis validation set 
# '''Minimizing over-fitting, improving model generalization while using maximum possible training data for fitting our model ''' 
# sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.2, random_state=SEED, ) 

# for i, (tr_ids, ts_ids) in enumerate(sss.split(X_train_full, y_train_full)): 
#     model = LGBMClassifier(random_state=SEED+i, n_estimators=600, learning_rate=0.05, metric='binary_logloss',) 
#     model = model.fit(X_train_full[tr_ids], y_train_full[tr_ids], early_stopping_rounds=50, eval_set=[(X_train_full[ts_ids], y_train_full[ts_ids])], verbose=0 ) 
#     # scoring 
#     print(i, 'best_iteration', model._best_iteration) # early stopping will retain best iteration weights to use for predictions  
#     print(i, 'log_loss', log_loss(y_train_full[ts_ids], model.predict_proba(X_train_full[ts_ids])[:,1]))
#     print(i, 'accuracy', model.score(X_train_full[ts_ids], y_train_full[ts_ids]), '\n')
#     print()

#     # predict on test set 
#     pred_dict['lgb_'+str(i)] = model.predict_proba(X_test)[:, 1] 
    
# predict_frame = pd.DataFrame(pred_dict)
# predict_frame['lgb_avg'] = predict_frame.iloc[:,-n_splits:].mean(axis=1) # soft voting gives better results than hard voting 
# # predict_frame['gb_avg'] = predict_frame.median(axis=1) # hard voting  
# print(predict_frame.describe()) 

# y_test_pred = np.round(predict_frame['lgb_avg'].values)  
# # y_test_pred = np.round(predict_frame['lgb_0'].values)  
# print(y_test_pred[:5]) 
# print(np.mean(y_test_pred)) 

# submission = pd.DataFrame(dict(PassengerId = test.PassengerId, Transported = y_test_pred)) 
# submission.Transported = submission.Transported.astype('bool')
# submission.to_csv('submission.csv', index=False)
# submission.head() 
    
# print('SEED:', SEED) # to see THE random seed used in this run 
# print ( ((predict_frame > 0.45) & (predict_frame < 0.55)).sum() )

# # print(model.__dict__)

In [20]:
# Tuned GradientBoostingClassifier with early stopping basis validation set 
'''Minimizing over-fitting, improving model generalization while using maximum possible training data for fitting our model ''' 
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=2, random_state=SEED, ) 

for i, (tr_ids, ts_ids) in enumerate(sss.split(X_train_full, y_train_full)): 
    # training on shuffled dataset with 0.2 validation_fraction 
    model = GradientBoostingClassifier(max_depth=7, max_features='sqrt', n_estimators=600, n_iter_no_change=50, tol=0, subsample=0.75, learning_rate=0.05, validation_fraction=0.2, verbose=0, random_state=SEED+i )
    model = model.fit(X_train_full[tr_ids], y_train_full[tr_ids] ) 
    # Checking OOB scores 
    raw_ests = model.n_estimators_ 
    opt_ests = np.argmax(np.cumsum(model.oob_improvement_)) + 1 
    estop_ests = raw_ests - model.n_iter_no_change 
#     print(opt_ests, raw_ests, sep=' / ' )
    print('Peak oob cum. improv: ', np.max(np.cumsum(model.oob_improvement_)), np.cumsum(model.oob_improvement_)[opt_ests-1] )
    print('Before n_est tuning', i, np.sum(model.oob_improvement_), sep=': ')
#     print(i, 'log_loss', log_loss(y_train_full[ts_ids], model.predict_proba(X_train_full[ts_ids])[:,1])) 
#     print(i, 'accuracy', model.score(X_train_full[ts_ids], y_train_full[ts_ids]), '\n') 
    
    ''' To refit the model using n_estimators corresponding to peak cum. improvement, use n_estimators=opt_ests '''
    
    # Refit the model using n_estimators = raw_ests - early stopping 
    model = GradientBoostingClassifier(max_depth=7, max_features='sqrt', n_estimators=estop_ests, n_iter_no_change=50, tol=0, subsample=0.75, learning_rate=0.05, validation_fraction=0.2, verbose=0, random_state=SEED+i) 
    model = model.fit(X_train_full[tr_ids], y_train_full[tr_ids]) 
    # Checking OOB scores 
    opt_ests = np.argmax(np.cumsum(model.oob_improvement_)) + 1 
#     print(opt_ests, model.n_estimators_, sep=' / ' )
    print('Peak oob cum. improv: ', np.max(np.cumsum(model.oob_improvement_)), np.cumsum(model.oob_improvement_)[opt_ests-1] )
    print('After n_est tuning', i, np.sum(model.oob_improvement_), sep=': ')
#     print(i, 'log_loss', log_loss(y_train_full[ts_ids], model.predict_proba(X_train_full[ts_ids])[:,1])) 
#     print(i, 'accuracy', model.score(X_train_full[ts_ids], y_train_full[ts_ids]), '\n') 

    # predict on test set using model trained on StratifiedShuffleSplit of training data with 0.2 validation_fraction 
    pred_dict['gbc_'+str(i)] = model.predict_proba(X_test)[:, 1] 
    
predict_frame = pd.DataFrame(pred_dict)
predict_frame['gb_avg'] = predict_frame.iloc[:,-n_splits:].mean(axis=1) # soft voting gives better results than hard voting 
# predict_frame['gb_avg'] = predict_frame.median(axis=1) # hard voting  
print(predict_frame.describe()) 

y_test_pred = np.round(predict_frame['gb_avg'].values)  
# y_test_pred = np.round(predict_frame[2].values)  
print(y_test_pred[:5]) 
print(np.mean(y_test_pred)) 

submission = pd.DataFrame(dict(PassengerId = test.PassengerId, Transported = y_test_pred)) 
submission.Transported = submission.Transported.astype('bool')
submission.to_csv('submission.csv', index=False)
submission.head() 

print('SEED:', SEED) # to see THE random seed used in this run 
print ( ((predict_frame > 0.45) & (predict_frame < 0.55)).sum() )
# print(model.__dict__) 


Peak oob cum. improv:  0.5496187840898323 0.5496187840898323
Before n_est tuning: 0: 0.5293990643590882
Peak oob cum. improv:  0.5496187840898323 0.5496187840898323
After n_est tuning: 0: 0.5406349575724206
Peak oob cum. improv:  0.5446580913601031 0.5446580913601031
Before n_est tuning: 1: 0.4983053493158465
Peak oob cum. improv:  0.5446580913601031 0.5446580913601031
After n_est tuning: 1: 0.5103436336579543
Peak oob cum. improv:  0.5446520325322043 0.5446520325322043
Before n_est tuning: 2: 0.5278349528219952
Peak oob cum. improv:  0.5446520325322043 0.5446520325322043
After n_est tuning: 2: 0.537587234546397
Peak oob cum. improv:  0.5361552155796631 0.5361552155796631
Before n_est tuning: 3: 0.49363342760165524
Peak oob cum. improv:  0.5361552155796631 0.5361552155796631
After n_est tuning: 3: 0.5090312418921212
Peak oob cum. improv:  0.5483452090187825 0.5483452090187825
Before n_est tuning: 4: 0.5364464524288339
Peak oob cum. improv:  0.5483452090187825 0.5483452090187825
After n

In [21]:
# Building Keras neural network 

''' In the next few code blocks, I have explored Keras deep learning algorithm to sovle our binary prediction problem. The tuned version 
of this model has, like LightGBM, produced almost similar results as our tuned GradientBoostingClassifier, but has almost always 
come a close 2nd or 3rd. Plus, this one takes about 10-15 minutes to run compared to just a few minutes for other models. That said, 
I really got to learn much about concepts such as dropout rate, batch normalization between iterations, how algorithms try to minimize loss 
functions, and how early stopping works, and much more while working on this keras model. And then I used those insights to further tune 
my GradientBoostingClassifier and LGBMClassifer for better performance. ''' 

' In the next few code blocks, I have explored Keras deep learning algorithm to sovle our binary prediction problem. The tuned version \nof this model has, like LightGBM, produced almost similar results as our tuned GradientBoostingClassifier, but has almost always \ncome a close 2nd or 3rd. Plus, this one takes about 10-15 minutes to run compared to just a few minutes for other models. That said, \nI really got to learn much about concepts such as dropout rate, batch normalization between iterations, how algorithms try to minimize loss \nfunctions, and how early stopping works, and much more while working on this keras model. And then I used those insights to further tune \nmy GradientBoostingClassifier and LGBMClassifer for better performance. '

In [22]:
# # Building keras deep learning model 

# from tensorflow.keras.models import Sequential 
# from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, LeakyReLU 
# from tensorflow.keras.optimizers import Adam, SGD 
# from tensorflow.keras.callbacks import EarlyStopping 
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier 
# # from scikeras.wrappers import KerasClassifier # this isn't working here 
# set_random_seed = SEED 

# input_shape = (X_train_full.shape[1], ) 

# # create function to return deep learning model 
# def create_model(input_shape=input_shape, n_layers=1, n_nodes=24, dropout_rate=0.3, 
#                  activation='sigmoid', optimizer=Adam, learning_rate=0.001): 
#     dropout_rate = dropout_rate / n_layers 
#     model = Sequential() 
#     model.add(BatchNormalization(input_shape=input_shape) ) # input layer outside loop 
#     for layer in range(n_layers):
#         model.add(Dense(units=n_nodes, activation=activation) ) 
#         model.add(Dropout(rate=dropout_rate)) 
#         model.add(BatchNormalization() ) 
#     model.add(Dense(units=1, activation='sigmoid'))	# output layer outside loop with sigmoid activation for binary classification 
#     model.compile(optimizer=optimizer(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['binary_accuracy'])
#     return model 


In [23]:
# # tuning our keras model hyper-parameters with RandomizedSearchCV
# set_random_seed = SEED 

# # Convert keras model into an estimator usable with sklearn RandomizedSearchCV 
# model = KerasClassifier(build_fn=create_model, verbose=1, ) 

# # providing training and test set indices from train_test_split run previously to be used for model validation in 
# # RandomizedSearchCV as well as a validation set for early stopping in each run of keras model 
# cv = [(tts_tr, tts_val)]

# # define parameters dictionary for hyperparameter tuning 
# param_dist = dict(n_layers=np.arange(1,4),n_nodes=np.arange(16, 33, 4), dropout_rate=[0.2, 0.3, 0.4], 
#                   activation=['relu', LeakyReLU(alpha=0.1), 'tanh'], # batch_size=[16, 32, 64], 
#                   optimizer=[Adam, SGD], learning_rate=[0.001, 0.01, 0.1], # epochs=np.arange(50, 201), 
#                   ) 
# # Instantiate RandomizedSearchCV 
# rscv = RandomizedSearchCV(estimator=model, param_distributions=param_dist, random_state=SEED, 
#                           n_iter=72, scoring='neg_log_loss', cv=cv, # n_jobs=-1, 
#                           verbose=2) 

# # define early stopping parameters 
# early_stopping = EarlyStopping(monitor='val_loss', patience=16, restore_best_weights=True, )

# # Run RandomizedSearchCV using training data & validation set for early stopping 
# rscv.fit(X_train_full, y_train_full, validation_data=(X_val, y_val), 
#          batch_size=32, epochs=150, callbacks=[early_stopping] ) 

# # Print results
# print('best params: ', rscv.best_params_, sep='\n') 
# print('best score: ', rscv.best_score_, sep='\n') 

# y_predprob_val = rscv.best_estimator_.predict_proba(X_val)[:,1] 
# val_scores = [log_loss(y_val, y_predprob_val), 
#               roc_auc_score(y_val, y_predprob_val), 
#               accuracy_score(y_val, np.round(y_predprob_val))] 

# print('best scores on val set: ', val_scores , sep='\n') 
# rscv_results = pd.DataFrame(rscv.cv_results_)
# # pd.set_option('display.max_rows', 75) 
# # print(rscv_results) 
# rscv_results.to_csv('rscv_results.csv')


In [24]:
''' In separate runs of RandomizedSearchCV with my keras model, I found following to be 
among the best performing model configurations, with the top one being the best of all: 

create_model(n_layers=1, n_nodes=16, dropout_rate=0.2, activation='tanh', optimizer=Adam, learning_rate=0.01,) 
create_model(n_layers=1, n_nodes=20, dropout_rate=0.2, activation='relu', optimizer=Adam, learning_rate=0.01,) 
create_model(n_layers=1, n_nodes=24, dropout_rate=0.2, activation='relu', optimizer=Adam, learning_rate=0.01,) 
create_modeln_layers=1, n_nodes=24, dropout_rate=0.3, activation='sigmoid', optimizer=Adam, learning_rate=0.001) 

create_model(n_layers=2, n_nodes=16, dropout_rate=0.3, activation='tanh', optimizer=Adam, learning_rate=0.01,) 
create_model(n_layers=2, n_nodes=20, dropout_rate=0.2, activation='relu', optimizer=Adam, learning_rate=0.01,) 
create_model(n_layers=2, n_nodes=24, dropout_rate=0.2, activation='relu', optimizer=Adam, learning_rate=0.01,) 

create_model(n_layers=3, n_nodes=20, dropout_rate=0.2, activation='relu', optimizer=Adam, learning_rate=0.01,) 
create_model(n_layers=3, n_nodes=24, dropout_rate=0.2, activation='relu', optimizer=Adam, learning_rate=0.01,) 
''' 

" In separate runs of RandomizedSearchCV with my keras model, I found following to be \namong the best performing model configurations, with the top one being the best of all: \n\ncreate_model(n_layers=1, n_nodes=16, dropout_rate=0.2, activation='tanh', optimizer=Adam, learning_rate=0.01,) \ncreate_model(n_layers=1, n_nodes=20, dropout_rate=0.2, activation='relu', optimizer=Adam, learning_rate=0.01,) \ncreate_model(n_layers=1, n_nodes=24, dropout_rate=0.2, activation='relu', optimizer=Adam, learning_rate=0.01,) \ncreate_modeln_layers=1, n_nodes=24, dropout_rate=0.3, activation='sigmoid', optimizer=Adam, learning_rate=0.001) \n\ncreate_model(n_layers=2, n_nodes=16, dropout_rate=0.3, activation='tanh', optimizer=Adam, learning_rate=0.01,) \ncreate_model(n_layers=2, n_nodes=20, dropout_rate=0.2, activation='relu', optimizer=Adam, learning_rate=0.01,) \ncreate_model(n_layers=2, n_nodes=24, dropout_rate=0.2, activation='relu', optimizer=Adam, learning_rate=0.01,) \n\ncreate_model(n_layers=3

In [25]:
# # Fitting multiple subsamples of training data on tuned keras neural network model to make predictions 

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED, ) 
# # skf = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.2, random_state=SEED, ) 
# early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True, )
# history_frame = pd.DataFrame() 
# # pred_dict = dict() 

# for i, (tr_ids, ts_ids) in enumerate(skf.split(X_train_full, y_train_full)): 
#     # training on cross-validation split 
#     set_random_seed = SEED + i 
#     model = create_model(n_layers=1, n_nodes=16, dropout_rate=0.2, activation='tanh', 
#                          optimizer=Adam, learning_rate=0.01,) 
#     history = model.fit(X_train_full[tr_ids], y_train_full[tr_ids], epochs=200, 
#                         batch_size=32, callbacks=[early_stopping], 
#                         validation_data=[X_train_full[ts_ids], y_train_full[ts_ids]]) 
#     history_frame = history_frame.append(pd.DataFrame({**history.history, 'split': i}), ignore_index=True) 
#     # predict on test set using model trained on cross-validated subset of training data 
#     pred_dict['nn_'+str(i)] = model.predict(X_test).flatten() 

# print(history_frame.nsmallest(20, 'val_loss'))
# print('Above values sorted by val_loss, below values sorted by val_accuracy')
# print(history_frame.nlargest(20, 'val_binary_accuracy'))

# predict_frame = pd.DataFrame(pred_dict)
# predict_frame['nn_avg'] = predict_frame.iloc[:, -skf.n_splits:].mean(axis=1) # final prediction submission using soft voting 
# predict_frame['nn_avg'] = predict_frame.iloc[:, -skf.n_splits:].median(axis=1) # submission using hard voting -> isn't good as soft voting 
# print(predict_frame.describe()) 

# y_test_pred = np.round(predict_frame['nn_avg'].values)  
# # y_test_pred = np.round(predict_frame[2].values)  
# print(y_test_pred[:5]) 
# print(np.mean(y_test_pred)) 

# submission = pd.DataFrame(dict(PassengerId = test.PassengerId, Transported = y_test_pred)) 
# submission.Transported = submission.Transported.astype('bool')
# submission.to_csv('submission.csv', index=False)
# submission.head() 


In [26]:
# '''Predicting basis soft voting on multiple algorithm predictions - LGBMClassifier, GradientBoostingClassifer, 
# and Keras NN when all are activated in previous code blocks '''
# '''kaggle best score for this block so far 0.8036 V31 '''

# predict_frame = pd.DataFrame(pred_dict)
# predict_frame['all_avg'] = predict_frame.mean(axis=1) 
# print(predict_frame.describe()) 

# y_test_pred = np.round(predict_frame['all_avg'].values)  
# # y_test_pred = np.round(predict_frame[2].values)  
# print(y_test_pred[:5]) 
# print(np.mean(y_test_pred)) 

# submission = pd.DataFrame(dict(PassengerId = test.PassengerId, Transported = y_test_pred)) 
# submission.Transported = submission.Transported.astype('bool')
# submission.to_csv('submission.csv', index=False)
# submission.head() 

# print('SEED:', SEED) # to see THE random seed used in this run 
# print ( ((predict_frame > 0.45) & (predict_frame < 0.55)).sum() ) 


In [27]:
# # Use this code block to generate a quick submission using any specified model 
# set_random_seed = SEED 
# # y_test_pred = # place model definition here 
# y_test_pred = GradientBoostingClassifier(learning_rate=0.05, max_depth=7, max_features='log2', n_estimators=154, 
#                                          subsample=0.55).fit(X_train_full, y_train_full).predict(X_test) # 0.80406 V15 
# print(np.mean(y_test_pred)) 
# submission = pd.DataFrame(dict(PassengerId = test.PassengerId, Transported = y_test_pred)) 
# submission.Transported = submission.Transported.astype('bool')
# submission.to_csv('submission.csv', index=False)
# submission.head() 
